In [ ]:
import pandas as pd
import datetime
from datetime import timedelta
def present_rate():
    b = []
    dataframe_list = []
    base_rate = pd.read_csv('/Users/stillssi/Desktop/BOK_TEAM_4/01_Crawling/base_rate/base_rate.csv')
    base_rate['date'] = pd.to_datetime(base_rate['date'])
    base_rate = base_rate.sort_values(by='date',ascending=True)
    base_rate.set_index('date', inplace=True)
    for i in range(len(base_rate.index)):
        start_date = pd.to_datetime(base_rate.index[i]) ## 시작 날짜
        try:
            tmp = {}
            end_date = pd.to_datetime(base_rate.index[i+1]) ## 마지막 날짜\
            bet = pd.date_range(start_date,end_date+ timedelta(days = -1),freq='D').values
            
            tmp = {'date': bet, 'rate': [base_rate.loc[start_date].rate]*len(bet)} 
            b = pd.DataFrame(tmp)
            b.set_index('date', inplace=True)
            dataframe_list.append(b)
        except:
            break
        
    merge_dataframe = pd.concat(dataframe_list)
    merge_dataframe = merge_dataframe.iloc[46:4100]
    merge_dataframe.to_csv('./base_rate.csv')
    return merge_dataframe

present_rate()

In [ ]:
import numpy as np
from datetime import datetime
from datetime import timedelta

def future_rate(fra):
    da_list, ra_list = [],[]
    base_r = fra
    for i in range(len(base_r.index)):
        if base_r.index[i] == datetime.strptime('2022-01-01', '%Y-%m-%d'):
            break
        da_list.append(base_r.index[i])
        ra_list.append(base_r.loc[base_r.index[i] + timedelta(days = 28)].rate)
        
    label_ = {'date': da_list, 'rate': ra_list}
    label_df = pd.DataFrame(label_)
    label_df.set_index('date', inplace=True)
    return label_df

pre_rate = present_rate()
fu_rate = future_rate(pre_rate)

In [ ]:
def please2(x):
    return 1 if x > 0.03 else -1 if x < -0.03 else 0
new_df = (pre_rate - fu_rate)
new_df['label'] = new_df['rate'].apply(lambda x : please2(x))

In [ ]:
new_df.to_csv('base_rate_label.csv')


In [ ]:
import itertools
import re
import os
from eKoNLPy.ekonlpy.sentiment import MPCK
import pandas as pd
from datetime import datetime

def minutes_ngram():
    '''
    의사록 5-gram 함수
    return: n-gram 개수
    '''
    i = 0
    mpck = MPCK()
    TXT_DIR = '//Users/stillssi/Desktop/minutes/Data/split_data/' #split text 파일 경로
    label_data = pd.read_csv('/Users/stillssi/Desktop/BOK_TEAM_4/02_Preprocessing/minutes_preprocessing/base_rate_label.csv')
    file_list = os.listdir(TXT_DIR)
    df = pd.DataFrame(columns=['date', 'ngrams'])
    label_data['date'] = pd.to_datetime(label_data['date'], infer_datetime_format=True)
    for file in file_list:

        ngram_list = []
        text = []
        try:
            f = open(TXT_DIR+file,'r', encoding='utf-8')
            day = datetime.strptime(file[:-4], '%Y.%m.%d')
            
            lines = f.readlines()

            hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
            
            for l in lines:
                
                result = hangul.sub('', l)
                result = result.split(' ')
                result_removed = [i for i in result if i != '']
                line = ' '.join(result_removed)
                text.append(line)

            if len(text) == 1:
                pass
            else:
                text = list(itertools.chain(*text))
            resultt = ''.join(text)
            tokens = mpck.tokenize(resultt)
            ngrams = mpck.ngramize(tokens)
            ngram_list.append(ngrams)

            ngram_list= list(itertools.chain(*ngram_list))
            df.loc[i]=[day,ngram_list]
            i = i+1    
    
        except Exception as e:
            pass
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    

    df_LEFT_JOIN = pd.merge(df, label_data, left_on='date', right_on='date', how='left')
    minutes_labeling_data = df_LEFT_JOIN.sort_values(by='date',ascending=True)
    minutes_labeling_data.set_index('date', inplace=True)
    minutes_labeling_data.sort_index(ascending=True, inplace=True)
    minutes_labeling_data.to_csv('./minutes_ngram_labeling.csv')
    return minutes_labeling_data




In [ ]:
min_ngram = minutes_ngram()
min_ngram

In [ ]:
min_ngram

In [ ]:
import nltk
import itertools

ngram_list = list(itertools.chain(*list(min_ngram['ngrams'])))
up_ngram_list = list(itertools.chain(*(list(min_ngram[min_ngram['label']==1]['ngrams']))))
down_ngram_list = list(itertools.chain(*(list(min_ngram[min_ngram['label']==-1]['ngrams']))))
same_ngram_list = list(itertools.chain(*(list(min_ngram[min_ngram['label']==0]['ngrams']))))

In [ ]:
fdist = nltk.FreqDist(ngram_list) 
count_df = pd.DataFrame(list(zip(fdist.keys(), fdist.values())))
count_df.columns = ['ngrams', 'n_sums']

up_fdist = nltk.FreqDist(up_ngram_list)
up_count_df = pd.DataFrame(list(zip(up_fdist.keys(), up_fdist.values())))
up_count_df.columns = ['ngrams', 'n_sums']

down_fdist = nltk.FreqDist(down_ngram_list)
down_count_df = pd.DataFrame(list(zip(down_fdist.keys(), down_fdist.values())))
down_count_df.columns = ['ngrams', 'n_sums']

In [ ]:
down_count_df.to_csv('./adsf.csv')


In [ ]:
count_df.columns = ['ngrams', 'n_sums']
count_df

In [ ]:
ngrams_df = pd.merge(left = count_df, right = up_count_df, how='left', on='ngrams')
print(ngrams_df) # x=count_df, y=up_count_df

In [ ]:
ngrams_df = pd.merge(left = ngrams_df, right = down_count_df, how='left', on='ngrams') # n_sums=down_count_df

In [ ]:
ngrams_df.columns = ['ngrams', 'n_count_df', 'n_up_count_df', 'n_down_count_df']

In [ ]:
for i in range(len(ngrams_df)):
    up_nbc = ngrams_df.loc[i, 'n_up_count_df'] / ngrams_df['n_up_count_df'].sum()
    down_nbc = ngrams_df.loc[i, 'n_down_count_df'] / ngrams_df['n_down_count_df'].sum()
    ngrams_df.loc[i, 'up_NBC'] = up_nbc
    ngrams_df.loc[i, 'down_NBC'] = down_nbc

In [ ]:
ngrams_df